# Modeling Notebook

This notebook demonstrates training and evaluating machine learning models for stress detection.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import sys
import os

# Add src to path
sys.path.append('../src')

# Import our modules
from models import StressDetectionModel, compare_models, leave_one_subject_out_cv
from utils import load_dataset

%matplotlib inline

## Load Processed Features

In [ ]:
# Load features
processed_dir = '../data/processed'
features_path = os.path.join(processed_dir, 'extracted_features.csv')

if os.path.exists(features_path):
    features_df = load_dataset(features_path)
    print("Features loaded successfully!")
    print(f"Features shape: {features_df.shape}")
    print(features_df.head())
else:
    print("Processed features not found. Creating sample data for demonstration.")
    
    # Create sample features for demonstration
    np.random.seed(42)
    n_samples = 100
    
    features_df = pd.DataFrame({
        'mean_rr': np.random.normal(800, 50, n_samples),
        'sdnn': np.random.normal(50, 10, n_samples),
        'rmssd': np.random.normal(30, 15, n_samples),
        'pnn50': np.random.normal(10, 5, n_samples),
        'lf_hf_ratio': np.random.normal(1.5, 0.5, n_samples),
        'mean_eda': np.random.normal(6, 2, n_samples),
        'std_eda': np.random.normal(1.5, 0.5, n_samples),
        'num_scr_peaks': np.random.poisson(5, n_samples),
        'mean_acc_magnitude': np.random.normal(10, 2, n_samples),
        'std_acc_magnitude': np.random.normal(1, 0.3, n_samples),
        'label': np.random.choice([0, 1], size=n_samples, p=[0.6, 0.4])  # 60% no stress, 40% stress
    })
    
    print("Sample features created successfully!")
    print(f"Features shape: {features_df.shape}")
    print(features_df.head())

## Prepare Data for Modeling

In [ ]:
# Separate features and labels
feature_columns = [col for col in features_df.columns if col not in ['label', 'window_id']]
X = features_df[feature_columns]
y = features_df['label']

print(f"Feature matrix shape: {X.shape}")
print(f"Label vector shape: {y.shape}")
print(f"Class distribution:\n{y.value_counts()}")

## Split Data

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
print(f"Training class distribution:\n{y_train.value_counts()}")
print(f"Testing class distribution:\n{y_test.value_counts()}")

## Train Individual Models

In [ ]:
# Train a Random Forest model
rf_model = StressDetectionModel('rf')
rf_model.train(X_train, y_train)

print("Random Forest model trained successfully!")

In [ ]:
# Evaluate the model
rf_metrics = rf_model.evaluate(X_test, y_test)
print("Random Forest Model Evaluation:")
print(f"Accuracy: {rf_metrics['accuracy']:.3f}")
print(f"Precision: {rf_metrics['precision']:.3f}")
print(f"Recall: {rf_metrics['recall']:.3f}")
print(f"F1-Score: {rf_metrics['f1_score']:.3f}")

if 'roc_auc' in rf_metrics:
    print(f"ROC AUC: {rf_metrics['roc_auc']:.3f}")

print(f"\nConfusion Matrix:\n{np.array(rf_metrics['confusion_matrix'])}")

## Compare Multiple Models

In [ ]:
# Compare different models
comparison_results = compare_models(X, y, test_size=0.2, random_state=42)

print("Model Comparison Results:")
print("=" * 50)

for model_name, metrics in comparison_results.items():
    print(f"\n{model_name}:")
    print(f"  Accuracy: {metrics['accuracy']:.3f}")
    print(f"  Precision: {metrics['precision']:.3f}")
    print(f"  Recall: {metrics['recall']:.3f}")
    print(f"  F1-Score: {metrics['f1_score']:.3f}")
    if 'roc_auc' in metrics:
        print(f"  ROC AUC: {metrics['roc_auc']:.3f}")

## Visualize Model Comparison

In [ ]:
# Create comparison visualization
model_names = list(comparison_results.keys())
metrics = ['accuracy', 'precision', 'recall', 'f1_score']

# Extract metric values
metric_values = {metric: [comparison_results[model][metric] for model in model_names] for metric in metrics}

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(model_names))
width = 0.2

for i, metric in enumerate(metrics):
    ax.bar(x + i*width, metric_values[metric], width, label=metric.capitalize())

ax.set_xlabel('Models')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x + width * 1.5)
ax.set_xticklabels(model_names)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Hyperparameter Tuning

In [ ]:
# Example hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

print("Performing hyperparameter tuning for Random Forest...")
rf_tuned = StressDetectionModel('rf')

# Note: This would take some time to run
# tuning_results = rf_tuned.hyperparameter_tuning(X_train, y_train, param_grid, cv=3)
# print(f"Best parameters: {tuning_results['best_params']}")
# print(f"Best cross-validation score: {tuning_results['best_score']:.3f}")

print("Hyperparameter tuning example shown (commented out for faster execution).")

## Leave-One-Subject-Out Cross-Validation

In [ ]:
# Example of leave-one-subject-out cross-validation
# This requires subject IDs for each sample
print("Leave-One-Subject-Out Cross-Validation Example:")

# Create artificial subject IDs for demonstration
subjects = np.random.choice(['S01', 'S02', 'S03', 'S04', 'S05'], size=len(X))
subject_ids = pd.Series(subjects)

print(f"Number of unique subjects: {subject_ids.nunique()}")
print(f"Subject distribution:\n{subject_ids.value_counts()}")

# Note: This would take some time to run
# loo_results = leave_one_subject_out_cv(X, y, subject_ids, model_type='rf')
# print(f"LOO CV Average Accuracy: {loo_results['average_scores']['accuracy']:.3f}")
# print(f"LOO CV Average F1-Score: {loo_results['average_scores']['f1_score']:.3f}")

print("Leave-one-subject-out CV example shown (commented out for faster execution).")

## Save Best Model

In [ ]:
# Save the best performing model
model_dir = '../models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# For demonstration, we'll save the RF model
# In practice, you would save the best performing model based on your evaluation
model_path = os.path.join(model_dir, 'stress_detection_rf.pkl')
# rf_model.save_model(model_path)
# print(f"Model saved to {model_path}")

print("Model saving example shown (commented out).")

## Feature Importance Analysis

In [ ]:
# Analyze feature importance from Random Forest
if hasattr(rf_model.model, 'feature_importances_'):
    importances = rf_model.model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': feature_columns,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    print("Top 10 Most Important Features:")
    print(feature_importance_df.head(10))
    
    # Plot feature importances
    plt.figure(figsize=(10, 6))
    top_features = feature_importance_df.head(10)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importance')
    plt.title('Top 10 Feature Importances (Random Forest)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## Summary

In this notebook, we have:
1. Loaded and prepared the feature data
2. Trained and evaluated individual models
3. Compared multiple machine learning algorithms
4. Demonstrated hyperparameter tuning
5. Showed cross-validation techniques
6. Analyzed feature importances

The next step would be to deploy the best performing model in a real-time application, which is demonstrated in the Streamlit app.